In [1]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import glob
import re
from mpl_toolkits import mplot3d
import os
from scipy.stats import rankdata
from sklearn import metrics
import math

%pylab inline
%matplotlib inline


Populating the interactive namespace from numpy and matplotlib


In [2]:
AllName = ["x1", "m1", "f100"]
NameTest = ["B2CL1_SORTCERY", "MCL1_SORTCERY", "B2LA1_SORTCERY"]
data = ["../SORTCERY_data/{}_merged.csv".format(n) for n in AllName]
binary_data = [
    "../SORTCERY_data/{}_binary_merged.csv".format(n) for n in AllName]


In [3]:
AA_to_int = {
    'A': 1,
    'ALA': 1,
    'C': 2,
    'CYS': 2,
    'D': 3,
    'ASP': 3,
    'E': 4,
    'GLU': 4,
    'F': 5,
    'PHE': 5,
    'G': 6,
    'GLY': 6,
    'H': 7,
    'HIS': 7,
    'I': 8,
    'ILE': 8,
    'K': 9,
    'LYS': 9,
    'L': 10,
    'LEU': 10,
    'M': 11,
    'MET': 11,
    'N': 12,
    'ASN': 12,
    'P': 13,
    'PRO': 13,
    'Q': 14,
    'GLN': 14,
    'R': 15,
    'ARG': 15,
    'S': 16,
    'SER': 16,
    'T': 17,
    'THR': 17,
    'V': 18,
    'VAL': 18,
    'W': 19,
    'TRP': 19,
    'Y': 20,
    'TYR': 20,
    'X': 21
}

AA_to_int = {key: val - 1 for key, val in AA_to_int.items()}


def seq_to_ints(sequence):
    return [AA_to_int[residue] for residue in sequence]


In [7]:
for n in AllName:
    df = pd.read_csv(f"../SORTCERY_data/{n}_merged.csv")
    df2 = df.sort_values(by=[f'{n}_mean_ener'])
    print(len(df2))
    print(df2[:10])
    print(df2[-10:])

    # peptides = {"peptides": np.array([seq_to_ints(x[2:22]) for x in df['protein']])} # as per Frappier notebooks, the peptides are shorter, length 20 for design
    # energies = {"energies": np.array(df[f'{n}_mean_ener'])}
    # with open(f"../SORTCERY_data/{n}_peptide_array.pickle", 'wb') as fp:
    #     pickle.dump(peptides, fp)
    # with open(f"../SORTCERY_data/{n}_energy_array.pickle", 'wb') as fp:
    #     pickle.dump(energies, fp)


4386
      Unnamed: 0                 protein  x1_mean_ener
297          297  GQWEREIGAHLRRVADLFNAQA    -12.683270
375          375  GQWIREIAANLRRIADLINAQF    -12.478250
367          367  GQWIREIAADLRRLADLFNAQL    -12.453862
904          904  GQWVREISASLRRIADLLNAQF    -12.415417
451          451  GQWIREISAILRRLADLFNAQF    -12.386538
384          384  GQWIREIAAYLRRIADQLNAQL    -12.360402
450          450  GQWIREISAILRRIADDFNAQI    -12.241185
458          458  GQWIREISARLRRLADDLNAQL    -12.217108
344          344  GQWEREVGAQLARIGDDLVEQC    -12.144957
1691        1691  GRPLIWLAQELARIADEFHTYN    -12.136484
      Unnamed: 0                 protein  x1_mean_ener
1040        1040  GRGEADITQELRRIGDEFQAYY     -9.167152
1010        1010  GRAELQAIQELIRIGDELDAYY     -9.123633
2302        2302  GRSEDQIVQELIRIGDEFNAYY     -9.106535
2296        2296  GRSEDEVTQELIRIGDEFEAYY     -9.088502
1314        1314  GRPEYEITQELGRIGDELDAYY     -9.031933
1209        1209  GRPEFHAVQELKRIGDESKAYY     -9.026778
1015 

In [50]:
# fractions = {"x1": 0.316, "m1": 0.347, "f100": 0.426}
fractions = {"x1": 1/3, "m1": 1/3, "f100": 1/3}
for n in AllName:
    df_original = pd.read_csv(f"../SORTCERY_data/{n}_merged.csv")
    df = df_original.sort_values(by=[f'{n}_mean_ener'])[
        int(len(df_original)*fractions[n]):]
    print(df.skew(axis=0, skipna=True))
    print(len(df_original), len(df_original)*(1-fractions[n]))

    # peptides = {"peptides": np.array([seq_to_ints(x[2:22]) for x in df['protein']])} # as per Frappier notebooks, the peptides are shorter, length 20 for design
    # energies = {"energies": np.array(df[f'{n}_mean_ener'])}
    # with open(f"../SORTCERY_data/{n}_peptide_array.pickle", 'wb') as fp:
    #     pickle.dump(peptides, fp)
    # with open(f"../SORTCERY_data/{n}_energy_array.pickle", 'wb') as fp:
    #     pickle.dump(energies, fp)


Unnamed: 0     -0.199402
x1_mean_ener    0.107969
dtype: float64
4386 2924.0000000000005
Unnamed: 0      0.272438
m1_mean_ener   -0.114502
dtype: float64
4491 2994.0000000000005
Unnamed: 0       -0.073798
f100_mean_ener   -0.460602
dtype: float64
3805 2536.666666666667


In [59]:
# fractions = {"x1": 0.316, "m1": 0.347, "f100": 0.426} # original fractions found manually to split the data into normally distributed train/validation/test splits, but did not lead to successful fine-tuning
fractions = {"x1": 0.3, "m1": 0.3, "f100": 0.3} # splits into 3 equal parts
for n in AllName:
    df_original = pd.read_csv(f"../SORTCERY_data/{n}_merged.csv")
    df = df_original.sort_values(by=[f'{n}_mean_ener'])

    sp = int(len(df_original)*fractions[n])
    df_train = df[sp:].sample(frac=1)
    # split in two to mitigate memory problems
    df_train1 = df_train[:len(df_train)//2]
    df_train2 = df_train[len(df_train)//2:]
    df_val = df[sp//2:sp].sample(frac=1)
    df_test = df[:sp//2].sample(frac=1)

    print(df_train1[:10], len(df_train1))
    print(df_train2[:10], len(df_train2))
    print(df_val[:10], len(df_val))
    print(df_test[:10], len(df_test))

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df_train1['protein']])}
    energies = {"energies": np.array(df_train1[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_train1.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_train1.pickle", 'wb') as fp:
        pickle.dump(energies, fp)

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df_train2['protein']])}
    energies = {"energies": np.array(df_train2[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_train2.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_train2.pickle", 'wb') as fp:
        pickle.dump(energies, fp)

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df_val['protein']])}
    energies = {"energies": np.array(df_val[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_val.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_val.pickle", 'wb') as fp:
        pickle.dump(energies, fp)

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df_test['protein']])}
    energies = {"energies": np.array(df_test[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_test.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_test.pickle", 'wb') as fp:
        pickle.dump(energies, fp)


      Unnamed: 0                 protein  x1_mean_ener
2490        2490  GRTEHDIIQELGRIGDEVDAYY     -9.456355
1109        1109  GRGEVDVVQELTRIGDEVEAYY     -9.484051
3530        3530  GQWQREICAFLRRSADDINAQI     -9.634392
3101        3101  GQWEREIGAQLARSGDDLVKQG     -9.660631
1999        1999  GRPVIWICQYLRRIGDMANAYF     -9.764449
478          478  GQWIREIYASLRRVADNYNAQF     -9.933329
3196        3196  GQWIREIGAQLARHGDDLLEQG     -9.635740
3990        3990  GRPIIWIYQGLRRLGDLANAYI     -9.490747
1932        1932  GRPVIWFAQELARRGDEFDEYG     -9.612266
2844        2844  GQRAVQVIAQLIRMADDVQAQY     -9.563331 1535
      Unnamed: 0                 protein  x1_mean_ener
1246        1246  GRPEIWICQFLRRFGDHFNAYS     -9.726150
1318        1318  GRPEYQIIQELERIGDETDAYY     -9.369128
3320        3320  GQWLREICACLRRCADMYNAQA     -9.788579
3949        3949  GRPIIWHAQELVRHADEFLAYN     -9.754917
3039        3039  GQWAVQAVAQLRRMADDIHAQY     -9.484753
2215        2215  GRRELQVIQELGRIGDEVKAYY     -9.513462
2185 

In [73]:
for n in AllName:
    df_original = pd.read_csv(f"../SORTCERY_data/{n}_merged.csv")
    df = df_original.sort_values(by=[f'{n}_mean_ener'])

    df_random = df.sample(frac=1)
    # split in three to mitigate memory problems
    df1 = df_random[:len(df_random)//3]
    df2 = df_random[len(df_random)//3:2*len(df_random)//3]
    df3 = df_random[2*len(df_random)//3:]
    print(len(df1), len(df2), len(df3))

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df1['protein']])}
    energies = {"energies": np.array(df1[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_split1.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_split1.pickle", 'wb') as fp:
        pickle.dump(energies, fp)

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df2['protein']])}
    energies = {"energies": np.array(df2[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_split2.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_split2.pickle", 'wb') as fp:
        pickle.dump(energies, fp)

    # as per Frappier notebooks, the peptides are shorter, length 20 for design
    peptides = {"peptides": np.array(
        [seq_to_ints(x[2:22]) for x in df3['protein']])}
    energies = {"energies": np.array(df3[f'{n}_mean_ener'])}
    with open(f"../SORTCERY_data/{n}_peptide_split3.pickle", 'wb') as fp:
        pickle.dump(peptides, fp)
    with open(f"../SORTCERY_data/{n}_energy_split3.pickle", 'wb') as fp:
        pickle.dump(energies, fp)


1462 1462 1462
1497 1497 1497
1268 1268 1269


In [15]:
for i in range(len(data)):
    d = pd.read_csv(data[i])
    b = pd.read_csv(binary_data[i])

    dlist = set(d.index)
    blist = set(b.index)
    print(len(dlist), len(blist))
    print(len(dlist - blist))
    print(len(blist - dlist))


4386 1882
2504
0
4491 1940
2551
0
3805 1686
2119
0


In [48]:
old_features = glob.glob("../../termless_sortcery_features/*/*.features")
for fname in old_features:
    with open(fname, 'rb') as fp:
        features = pickle.load(fp)
        complex = os.path.basename(fname)
        if features['chain_lens'][1] != 20:
            print(fname, features['chain_lens'])


../../termless_sortcery_features\B2CL1_HUMAN_1BXL_A_holo\B2CL1_HUMAN_1BXL_A_holo.features [170, 16]
../../termless_sortcery_features\B2CL1_HUMAN_2YQ7_A_holo\B2CL1_HUMAN_2YQ7_A_holo.features [140, 17]
../../termless_sortcery_features\B2CL1_HUMAN_3IO8_C_holo\B2CL1_HUMAN_3IO8_C_holo.features [141, 18]
../../termless_sortcery_features\B2CL1_HUMAN_3R85_A_holo\B2CL1_HUMAN_3R85_A_holo.features [138, 19]
../../termless_sortcery_features\B2CL1_HUMAN_3R85_B_holo\B2CL1_HUMAN_3R85_B_holo.features [138, 17]
../../termless_sortcery_features\B2CL1_HUMAN_3R85_C_holo\B2CL1_HUMAN_3R85_C_holo.features [139, 18]
../../termless_sortcery_features\B2CL1_HUMAN_3R85_D_holo\B2CL1_HUMAN_3R85_D_holo.features [139, 17]
../../termless_sortcery_features\B2CL1_HUMAN_4CIN_B_holo\B2CL1_HUMAN_4CIN_B_holo.features [134, 19]
../../termless_sortcery_features\B2CL1_HUMAN_5B1Z_A_holo\B2CL1_HUMAN_5B1Z_A_holo.features [144, 16]
../../termless_sortcery_features\B2CL1_HUMAN_5B1Z_B_holo\B2CL1_HUMAN_5B1Z_B_holo.features [144, 17]


In [87]:
# add SORTCERY data to feature files for input to models

bcl2_map = {"B2CL1": "x1", "MCL1": "m1", "B2LA1": "f100"}
old_features = glob.glob("../../termless_sortcery_features/*/*.features")

all_peptides = {}
all_energies = {}
for code in bcl2_map.values():
    with open(f"../SORTCERY_data/{code}_peptide_array.pickle", 'rb') as fpep:
        peps = pickle.load(fpep)
        all_peptides[code] = peps["peptides"]
    with open(f"../SORTCERY_data/{code}_energy_array.pickle", 'rb') as fener:
        eners = pickle.load(fener)
        all_energies[code] = eners["energies"]

for fname in old_features:
    with open(fname, 'rb') as fp:
        features = pickle.load(fp)
        complex = os.path.basename(fname)
        complex_name = complex[:complex.index('.')]
        family = complex.split("_")[0]
        family_code = bcl2_map[family]

        features["sortcery_seqs"] = all_peptides[family_code]
        features["sortcery_nrgs"] = all_energies[family_code]
        new_features = features

        new_fname = f"../../termless_features_with_sortcery/{complex_name}/{complex_name}.features"
        with open(new_fname, 'wb') as fp2:
            pickle.dump(new_features, fp2)


In [89]:
new_features = glob.glob("../../termless_features_with_sortcery/*/*.features")
for fname in new_features:
    with open(fname, 'rb') as fp:
        features = pickle.load(fp)
        print(os.path.basename(fname))
        print(features["sortcery_seqs"][0])
        print(features["sortcery_nrgs"][0])


B2CL1_HUMAN_1G5J_A_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2M04_A_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2P1L_A_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2P1L_C_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2P1L_E_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2P1L_G_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_2PON_B_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_3FDL_A_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 19 11  0 13 19]
-11.742132056109387
B2CL1_HUMAN_3IO8_A_holo.features
[ 0  0  0  2 17  5  0 13  9 14 14 10  0  2  2 1

In [60]:
bcl2_map = {"B2CL1": "x1", "MCL1": "m1", "B2LA1": "f100"}
old_features = glob.glob("../../termless_sortcery_features/*/*.features")

for split in ["train1", "train2", "val", "test"]:

    all_peptides = {}
    all_energies = {}
    for code in bcl2_map.values():
        with open(f"../SORTCERY_data/{code}_peptide_{split}.pickle", 'rb') as fpep:
            peps = pickle.load(fpep)
            all_peptides[code] = peps["peptides"]
        with open(f"../SORTCERY_data/{code}_energy_{split}.pickle", 'rb') as fener:
            eners = pickle.load(fener)
            all_energies[code] = eners["energies"]

    for fname in old_features:
        with open(fname, 'rb') as fp:
            features = pickle.load(fp)
            complex = os.path.basename(fname)
            complex_name = complex[:complex.index('.')]
            family = complex.split("_")[0]
            family_code = bcl2_map[family]

            features["sortcery_seqs"] = all_peptides[family_code]
            features["sortcery_nrgs"] = all_energies[family_code]
            new_features = features

            new_fname = f"../../termless_features_with_sortcery/{split}/{complex_name}/{complex_name}.features"
            with open(new_fname, 'wb') as fp2:
                pickle.dump(new_features, fp2)


In [61]:
new_features = glob.glob(
    "../../termless_features_with_sortcery/train1/*/*.features")
for fname in new_features:
    with open(fname, 'rb') as fp:
        features = pickle.load(fp)
        print(os.path.basename(fname))
        print(features["sortcery_seqs"][0])
        print(features["sortcery_nrgs"][0])


B2CL1_HUMAN_1G5J_A_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2M04_A_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2P1L_A_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2P1L_C_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2P1L_E_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2P1L_G_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_2PON_B_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_3FDL_A_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 19 19]
-9.456355386448497
B2CL1_HUMAN_3IO8_A_holo.features
[16  3  6  2  7  7 13  3  9  5 14  7  5  2  3 17  2  0 

In [72]:
import random

# generate random list so training inputs are randomized

splits = ["train", "val", "test"]
for s in splits:
    x = glob.glob(f"../../termless_features_with_sortcery/split/{s}*_holo")
    random.shuffle(x)
    for f in x:
        print(os.path.basename(f))


train1_MCL1_HUMAN_5C3F_A_holo
train2_MCL1_HUMAN_5C6H_E_holo
train2_MCL1_HUMAN_5UUM_B_holo
train2_MCL1_HUMAN_2KBW_A_holo
train1_B2CL1_HUMAN_4QVE_A_holo
train1_MCL1_HUMAN_5UUM_B_holo
train2_B2CL1_HUMAN_2P1L_E_holo
train1_MCL1_HUMAN_5C6H_O_holo
train2_MCL1_HUMAN_5C6H_I_holo
train1_MCL1_HUMAN_2NL9_A_holo
train2_B2CL1_HUMAN_4CIN_A_holo
train1_MCL1_HUMAN_5C6H_G_holo
train1_B2LA1_HUMAN_5UUL_A_holo
train1_MCL1_HUMAN_5C6H_E_holo
train2_MCL1_HUMAN_5C6H_G_holo
train2_B2CL1_HUMAN_1G5J_A_holo
train1_MCL1_HUMAN_2PQK_A_holo
train1_B2CL1_HUMAN_2PON_B_holo
train2_B2CL1_HUMAN_3PL7_A_holo
train2_MCL1_HUMAN_3PK1_A_holo
train2_MCL1_HUMAN_5C6H_W_holo
train2_MCL1_HUMAN_5C6H_M_holo
train1_B2CL1_HUMAN_3FDL_A_holo
train1_B2CL1_HUMAN_1G5J_A_holo
train1_MCL1_HUMAN_5C6H_A_holo
train1_B2LA1_HUMAN_4ZEQ_A_holo
train1_B2LA1_HUMAN_5UUK_A_holo
train2_B2CL1_HUMAN_4HNJ_A_holo
train2_MCL1_HUMAN_5C6H_A_holo
train1_B2CL1_HUMAN_4QVF_A_holo
train1_B2LA1_HUMAN_2VM6_A_holo
train1_MCL1_HUMAN_5C6H_Q_holo
train2_MCL1_HUMAN_5C6H_U_h

In [77]:
bcl2_map = {"B2CL1": "x1", "MCL1": "m1", "B2LA1": "f100"}
old_features = glob.glob("../../termless_sortcery_features/*/*.features")

for split in ["split1", "split2", "split3"]:

    all_peptides = {}
    all_energies = {}
    for code in bcl2_map.values():
        with open(f"../SORTCERY_data/{code}_peptide_{split}.pickle", 'rb') as fpep:
            peps = pickle.load(fpep)
            all_peptides[code] = peps["peptides"]
        with open(f"../SORTCERY_data/{code}_energy_{split}.pickle", 'rb') as fener:
            eners = pickle.load(fener)
            all_energies[code] = eners["energies"]

    for fname in old_features:
        with open(fname, 'rb') as fp:
            features = pickle.load(fp)
            complex = os.path.basename(fname)
            complex_name = complex[:complex.index('.')]
            family = complex.split("_")[0]
            family_code = bcl2_map[family]

            features["sortcery_seqs"] = all_peptides[family_code]
            features["sortcery_nrgs"] = all_energies[family_code]
            new_features = features

            new_fname = f"../../termless_features_with_sortcery/{split}_{complex_name}/{split}_{complex_name}"
            dirname = os.path.dirname(new_fname)
            if not os.path.exists(dirname):
                os.makedirs(dirname)
            with open(new_fname + '.features', 'wb') as fp2:
                pickle.dump(new_features, fp2)
            with open(new_fname + '.length', 'w') as fp:
                fp.write(str(1) + '\n')
                fp.write(str(features['seq_len']))

In [79]:
import random

splits = ["MCL1", "B2CL1", "B2LA1"]
for s in splits:
    x = glob.glob(f"../../termless_features_with_sortcery/combined_split3/*_{s}_*")
    random.shuffle(x)
    for f in x:
        print(os.path.basename(f))

split3_MCL1_HUMAN_3KJ0_A_holo
split3_MCL1_HUMAN_5C6H_M_holo
split1_MCL1_HUMAN_3D7V_A_holo
split2_MCL1_HUMAN_5C6H_M_holo
split3_MCL1_HUMAN_2KBW_A_holo
split1_MCL1_HUMAN_3PK1_C_holo
split1_MCL1_HUMAN_5UUM_B_holo
split3_MCL1_HUMAN_3D7V_A_holo
split1_MCL1_HUMAN_3KZ0_A_holo
split3_MCL1_HUMAN_5UUM_A_holo
split2_MCL1_HUMAN_5C6H_C_holo
split1_MCL1_HUMAN_5C6H_W_holo
split2_MCL1_HUMAN_3KZ0_A_holo
split1_MCL1_HUMAN_5C6H_C_holo
split2_MCL1_HUMAN_3IO9_A_holo
split1_MCL1_HUMAN_5C3F_A_holo
split2_MCL1_HUMAN_3PK1_A_holo
split3_MCL1_HUMAN_5C6H_O_holo
split3_MCL1_HUMAN_3KJ2_A_holo
split3_MCL1_HUMAN_3KZ0_B_holo
split2_MCL1_HUMAN_5C6H_Q_holo
split3_MCL1_HUMAN_5C6H_Q_holo
split1_MCL1_HUMAN_5C6H_O_holo
split3_MCL1_HUMAN_5C6H_K_holo
split2_MCL1_HUMAN_5C6H_G_holo
split2_MCL1_HUMAN_5C6H_K_holo
split3_MCL1_HUMAN_3KZ0_A_holo
split2_MCL1_HUMAN_3KJ0_A_holo
split2_MCL1_HUMAN_3PK1_C_holo
split1_MCL1_HUMAN_3KZ0_B_holo
split1_MCL1_HUMAN_2NL9_A_holo
split3_MCL1_HUMAN_3PK1_A_holo
split3_MCL1_HUMAN_5C6H_W_holo
split1_MCL